# La Prépa 1A 2023-2024
# Informatique
# TPL
#### MAMOUN Salma
#### SONIER Clementine
#### Groupe : C

## Partie 1 : Révisions et compléments de Python

### Construction de chaînes à partir de listes

In [1]:
couleurs = ['bleu', 'jaune', 'vert']

def affiche_couleurs(tableau):
    print('Les couleurs sont: ', end='')
    for i, couleur in enumerate(tableau) :                  
        if i != len(tableau)-1 :        # Boucle qui sert à afficher une virgule aprés chaque élément de la liste sauf le dernier
            print(couleur, end=', ')
        else:                                  
            print(couleur, end='')
    print('.')

In [2]:
print("QUESTION 1 -- Affichage des couleurs :")
affiche_couleurs(couleurs)

QUESTION 1 -- Affichage des couleurs :
Les couleurs sont: bleu, jaune, vert.


In [3]:
print("Tentative d'affectation non pertinente:")
chaine = affiche_couleurs(couleurs)
print("L'affectation est faite, voici le résultat :")
print(chaine)

Tentative d'affectation non pertinente:
Les couleurs sont: bleu, jaune, vert.
L'affectation est faite, voici le résultat :
None


In [4]:
# Autres tests
couleurs2 = ['bleu', 'bleu', 'vert']
print(affiche_couleurs(couleurs2))
couleurs3 = ['rouge', 'vert', 'bleu', 'jaune']
print(affiche_couleurs(couleurs3))

Les couleurs sont: bleu, bleu, vert.
None
Les couleurs sont: rouge, vert, bleu, jaune.
None


In [3]:
print("QUESTION 2 -- ")
def construit_chaine_couleurs(tableau):
    resultat = 'Les couleurs sont : '
    for i, couleur in enumerate(tableau) : 
         if i != len(tableau)-1 :
            resultat += couleur + ', ' # Boucle qui sert à afficher une virgule aprés chaque élément de la liste sauf le dernier
         else : 
            resultat += couleur + '.'
    return resultat

QUESTION 2 -- 


In [4]:
# Tests de construit_chaine_couleur
couleurs2 = ['bleu', 'bleu', 'vert']
print(construit_chaine_couleurs(couleurs2))
couleurs3 = ['rouge', 'vert', 'bleu', 'jaune']
print(construit_chaine_couleurs(couleurs3))

Les couleurs sont : bleu, bleu, vert.
Les couleurs sont : rouge, vert, bleu, jaune.


### Compléments sur Python : les tuples

In [5]:
print("SECTION 1.2 -- La même chose avec des tuples :")
couleurs_tuple = ('bleu', 'jaune', 'vert')
affiche_couleurs(couleurs_tuple)
print(construit_chaine_couleurs(couleurs_tuple))

SECTION 1.2 -- La même chose avec des tuples :
Les couleurs sont: bleu, jaune, vert.
Les couleurs sont : bleu, jaune, vert.


### Listes de tuples

In [8]:
contenu = [
    ('stylo', 'bleu', False),
    ('gomme', 'rouge', True),
    ('feutre', 'vert', False)
]

print('contenu = ', contenu)
print('contenu[0] = ', '(stylo, bleu, False)') 
print('contenu[1][2] = ', 'True') 
print('contenu[2][1][0] = ', 'v')

contenu =  [('stylo', 'bleu', False), ('gomme', 'rouge', True), ('feutre', 'vert', False)]
contenu[0] =  (stylo, bleu, False)
contenu[1][2] =  True
contenu[2][1][0] =  v


In [9]:
def description_trousse(contenu):
    resultat = ''
    for i, objet in enumerate(contenu) :  
        if objet[2] : # Boucle parcourue si l'objet est féminin (càd si objet[2] = True ) 
            resultat += '- une ' + objet[0] + ' ' +  objet[1]  # Affiche l'article 'une' si l'élément est féminin
        else :
            resultat += '- un ' + objet[0] + ' ' + objet[1]   # Affiche l'article 'un' si l'élément est masculin
        if i != len(contenu)-1 :
            resultat = resultat + '\n'
    return 'Dans ma trousse, il y a : \n' + resultat

In [10]:
# Test
print(description_trousse(contenu))

Dans ma trousse, il y a : 
- un stylo bleu
- une gomme rouge
- un feutre vert


## Partie 2 : interaction avec la base de données MusicBrainz

On commence par définir quelques cellules contenant le code 
pour utiliser la base de données.

In [32]:
import sqlite3
import os, sys

connection = None
dbfile = "musicbrainz-FR.db"

In [33]:
def set_connection(fn):
    def wrapped(dbfile, *args, **kwargs):
        global connection
        if not connection:
            init_connection(dbfile)
        return fn(connection, *args, **kwargs)
    return wrapped


In [34]:
OOM_HINT = ("Vérifiez que vous n'avez pas oublié une condition dans une jointure.\n"
            "Si le problème persite, essayez de relancer l'exécuteur de requêtes.\n")


In [35]:
#########################################################
# Cette fonction exécute sur la base
# la requête passée en paramètre
#########################################################
@set_connection
def execute_query(connection, query):
    try:
        # On récupère un objet curseur qui permet de parcourir 
        # l'ensemble résultat à la manière d'un itérateur.
        cursor = connection.cursor()

        # On exécute la requête ici.
        cursor.execute(query)

        # Après exécution de la requête, on récupère la réponse 
        # du SGBD et on renvoie le tout.
        return cursor.fetchall()
    except MemoryError:
        # Tester avec par exemple 
        # SELECT a1.* FROM Artistes AS a1, Artistes AS a2;
        print("Pas assez de mémoire pour exécuter la requête SQL.\n" + OOM_HINT)
        raise
    except sqlite3.Error as e:
        if len(e.args) > 0:
            msg = e.args[0]
        else:
            # Il arrive qu'on obtienne sqlite3.OperationalError sur un
            # "out of memory", et quand c'est le cas, e.args est un
            # tableau vide donc le e.args[0] lancerait une exception
            # dans le traitant d'exception :-(. On devine le diagnostique.
            msg = ("Erreur pendant l'exécution de la requête.\n" +
                   "Cette erreur peut se produire s'il n'y a pas " + 
                   "assez de mémoire.\n" + OOM_HINT)
        print(msg)
        raise

In [36]:
#########################################################
# Cette fonction initialise la connexion à la base
#########################################################
def init_connection(dbfile):
    global connection

    if not os.path.isfile(dbfile):
        print("Impossible d'ouvrir la base de données %s:\n"
              "le fichier n'existe pas." %
              dbfile)
        sys.exit(1)

    try:
        connection = sqlite3.connect(dbfile)

    except sqlite3.Error as e:
        print("Database connexion error - %s:" % e.args[0])
        close_connection()


In [37]:
#########################################################
# Cette fonction ferme la connexion à la base
#########################################################
def close_connection():
    global connection

    if connection:
        connection.close()
        connection = None


In [38]:
# Un test pour voir si tout fonctionne : la première requête du TP
execute_query(dbfile, 
                "SELECT DISTINCT nomDisque " + 
                "FROM Disques WHERE idArtiste = 22164;")

[('En concert',),
 ('Suppléments de mensonge',),
 ('Paris-Zénith',),
 ('Le Bonheur de la tentation',),
 ('Chroniques bluesymentales',),
 ('Scandale mélancolique Tour',),
 ('Guichet 102',),
 ('Alambic/Sortie-Sud',),
 ('1978-1983',),
 ('Live au Bataclan',),
 ('Météo für Nada',),
 ("De l'amour, de l'art ou du cochon ?",),
 ('Le Touquet juillet 1925',),
 ('Homo Plebis Ultimae Tour',),
 ('Thiéfaine 84.88 : Enregistrements Originaux',),
 ('En concert à Bercy',),
 ('Routes 88',),
 ("…tout corps vivant branché sur le secteur étant appelé à s'émouvoir…",),
 ('En concert, Volume 2',),
 ('Autorisation de délirer',),
 ('78-98',),
 ('Défloration 13',),
 ('Scandale mélancolique',),
 ("Fragments d'hébétude",),
 ('La Tentation du bonheur',),
 ('Séquelles',),
 ('Soleil cherche futur',),
 ('Dernières balises (avant mutation)',)]

In [41]:
# QUESTION 5
#def get_matching_artists(pattern):
    #query = "SELECT DISTINCT idArtiste, nomArtiste " + "FROM Artistes WHERE nomArtiste LIKE '%" +pattern+ "%';"
    #resultat0 = execute_query(dbfile, query)
    #resultat_ordonnee = sorted(resultat0, key=lambda x: x[1]) # trie des nomArtiste par ordre alphapétique
    #return resultat_ordonnee 

In [ ]:
## Remarque ## Bof c'était pas nécéssair !!
# On pouvait pas séléctionner et ordonner dans la même requête un prob de la sorte "You can only execute one statement at a time"
# Pour le résoudre on a décomposé notre code et on a utiliser la fct 'sorted' qui sert au trie des éléments d'une liste
# Le 'key=lambda x: x[1]' indique à python de trier le 2ème élément de chaque tuple (qui correspend bien au nom d'artiste)

In [49]:
def get_matching_artists(pattern):
    # Requête SQL qui cherche les identifiants d’artistes et noms d’artistes dont le nom contient le paramètre
    query = "SELECT DISTINCT idArtiste, nomArtiste " + "FROM Artistes WHERE nomArtiste LIKE '%" +pattern+ "%'ORDER BY nomArtiste"
    return execute_query(dbfile, query)

In [50]:
# Test
print(get_matching_artists("Python"))

[(619093, 'Lhacene 357 python'), (58603, 'Python Lee Jackson'), (331160, 'Spittin Python')]


In [51]:
# QUESTION 6
def test_get_matching_artists():
    nomArtiste = str(input("Nous cherchons:")) #utilisation du 'input' pour demander à l'utilisateur de rentrer ce qu'il cherche
    print (get_matching_artists(nomArtiste))

In [52]:
# Test
test_get_matching_artists()

Nous cherchons: Dalida


[(2011, 'Dalida'), (1287616, 'Dalida'), (805768, 'Dalida & Alain Delon')]


In [53]:
# QUESTION 7
def get_releases_by_artist(id_artiste):
    id_artiste = str(id_artiste) #création d'une chaine de caractères
    return execute_query(dbfile, 
            "SELECT DISTINCT nomDisque " + "FROM Disques WHERE idArtiste= "+id_artiste+ ";") 
# Requête SQL qui cherche le nom du disque avec le paramètre idArtiste

In [54]:
# Test
print(get_releases_by_artist(2011))

[("Le visage de l'amour",), ('Le temps des fleurs',), ('Les Années Orlando, Volume 9: Mourir sur scène',), ('Les Années Orlando, Volume 8: Fini, la comédie',), ("Les Années Orlando, Volume 1: La Rose que j'aimais",), ('Fini, La Comédie / Marjolaine',), ('Gondolier',), ('Les 50 Plus Belles Chansons',), ("Live : Instants d'émotions...",), ('Les Années Orlando, Volume 5: Bésame mucho',), ("L'An 2005",), ('Révolution',), ('Les Années Orlando, Volume 11: Pour en arriver là',), ('La Tumba',), ("Les Années Orlando, Volume 4: J'attendrai",), ("Les Années Orlando, Volume 3: Des gens qu'on aimerait connaître",), ("Les Années Orlando, Volume 10: Pour te dire je t'aime",), ("Je t'appelle encore",), ('Le Rêve oriental',), ('Talents, Volume 2',), ('The Glamourous Dalida',), ('Les Années Orlando',), ('Sus más grandes éxitos en español',), ("Paroles d'amour",), ("Paroles d'ailleurs",), ('Les Années Orlando, Volume 7: Laissez-moi danser',), ('Les Années Orlando, Volume 2: Une vie',), ('Dalida chante le

In [55]:
def main0(liste) : # Fonction qui sert à énumerer les éléments d'une liste
    artistes_2 = ''
    for i in range(len(liste)):
        artistes_2 += f'{i+1}  {liste[i]}\n'
    return artistes_2           
# Petit test
d = ['Indochine', 'Bernard Lavillier', 'Le Sud']
print(main0(d))

1  Indochine
2  Bernard Lavillier
3  Le Sud



In [71]:
def main1() :
    a = str(input('Quel artiste vous intéresse ?')) 
    artistes = get_matching_artists(a) # Liste des artistes avec leurs id qui correspondent au nom entré
    while not artistes : # la boucle est parcouru tant que la liste artistes est non vide
        a = str(input("Entrez le nom d'un autre artiste s'il vous plaît! "))
        artistes = get_matching_artists(a)
    resultat1 = '>' + a
    resultat2 = str( len(artistes)) + ' artiste(s) trouvé(s)...'
    print(resultat1)
    print(resultat2)
    print(main0(artistes))
    
    b = int(input(f"Parmi cette liste d’artistes, lequel vous convient (donnez le numéro entre 1 et {len(artistes)}) ?")) 
    c = artistes[b-1][0] # selectione id de l'artiste choisi
    disques = get_releases_by_artist(c)
    resultat3 = '>' + str(b)
    resultat4 = str( len(disques)) + ' disques(s) trouvé(s)...'
    print(resultat3)
    print(resultat4)
    print(main0(disques)) 

In [72]:
# Test
main1()

Quel artiste vous intéresse ? Dalida


>Dalida
3 artiste(s) trouvé(s)...
1  (2011, 'Dalida')
2  (1287616, 'Dalida')
3  (805768, 'Dalida & Alain Delon')



Parmi cette liste d’artistes, lequel vous convient (donnez le numéro entre 1 et 3) ? 1


>1
50 disques(s) trouvé(s)...
1  ("Le visage de l'amour",)
2  ('Le temps des fleurs',)
3  ('Les Années Orlando, Volume 9: Mourir sur scène',)
4  ('Les Années Orlando, Volume 8: Fini, la comédie',)
5  ("Les Années Orlando, Volume 1: La Rose que j'aimais",)
6  ('Fini, La Comédie / Marjolaine',)
7  ('Gondolier',)
8  ('Les 50 Plus Belles Chansons',)
9  ("Live : Instants d'émotions...",)
10  ('Les Années Orlando, Volume 5: Bésame mucho',)
11  ("L'An 2005",)
12  ('Révolution',)
13  ('Les Années Orlando, Volume 11: Pour en arriver là',)
14  ('La Tumba',)
15  ("Les Années Orlando, Volume 4: J'attendrai",)
16  ("Les Années Orlando, Volume 3: Des gens qu'on aimerait connaître",)
17  ("Les Années Orlando, Volume 10: Pour te dire je t'aime",)
18  ("Je t'appelle encore",)
19  ('Le Rêve oriental',)
20  ('Talents, Volume 2',)
21  ('The Glamourous Dalida',)
22  ('Les Années Orlando',)
23  ('Sus más grandes éxitos en español',)
24  ("Paroles d'amour",)
25  ("Paroles d'ailleurs",)
26  ('Les Années Orlan

### MusicBrainz Fights

In [61]:
# QUESTION 9
def rank_artists(idArtisteListe):
    id_str = ', '.join(str(id) for id in idArtisteListe)
    query = f"""
        SELECT nomArtiste , COUNT(DISTINCT idMorceau)  AS count_records
        FROM Artistes
        JOIN Morceaux ON Morceaux.idArtiste = Artistes.idArtiste
        WHERE Artistes.idArtiste IN ({id_str}) 
        GROUP BY nomArtiste
        ORDER BY count_records DESC
    """
    resultat = execute_query(dbfile, query)
    return resultat

In [62]:
print(rank_artists([1128, 685411]))

[('Jacques Brel', 425), ('Stromae', 17)]


In [94]:
def main2() :
    n = int(input("Entrez le nombre d'artistes que vous voulez comparez"))
    idArtisteListe = []
    for i in range(n) :
        nomArtiste = input("Entrez le nom d'un artiste : ")
        artiste = get_matching_artists(nomArtiste)
        if artiste:
            print(artiste)
            resultat_recherche = int(input(f"Parmi cette liste, lequel vous intéresse (donnez le numéro entre 0 et {len(artiste)-1})"))
            # 0 correspend au 1er element affiché , 1  au 2ème , ...
        idArtisteListe.append(artiste[resultat_recherche-1][0])
    
    if idArtisteListe:
        resultat = rank_artists(idArtisteListe)
        print("Résultats de la comparaison :")
        for i,artist in enumerate(resultat) :
                artiste = artist[0]
                count = artist[1]
                print(f"{artiste}: {count} enregistrements")  

In [95]:
# Le code ne traite pas le cas ou il n ya pas de correspandants du nom entré
# De plus il affiche des truc qu'on a pas besoin qu'ils soient affichés 

In [96]:
# Test
main2()

Entrez le nombre d'artistes que vous voulez comparez 2
Entrez le nom d'un artiste :  Dalida


[(2011, 'Dalida'), (1287616, 'Dalida'), (805768, 'Dalida & Alain Delon')]


Parmi cette liste, lequel vous intéresse (donnez le numéro entre 0 et 2) 0
Entrez le nom d'un artiste :  Dalida


[(2011, 'Dalida'), (1287616, 'Dalida'), (805768, 'Dalida & Alain Delon')]


Parmi cette liste, lequel vous intéresse (donnez le numéro entre 0 et 2) 2


Résultats de la comparaison :
Dalida: 2 enregistrements
Dalida & Alain Delon: 1 enregistrements


### Bilan